Load HyperParameters

In [1]:
from neat_src import loadHyp, updateHyp
from domain import load_task
from neat_src import DataGatherer, Neat 
from domain.config import games
from domain.task_gym import GymTask

game = games["slimevolley"]
task = GymTask(game)

hyp_default = 'p/default_neat.json'
hyp_adjust = "p/volley.json"
fileName = "volley"

hyp = loadHyp(pFileName=hyp_default, load_task=load_task)
updateHyp(hyp,load_task,hyp_adjust)

# neat = Neat(hyp)

	*** Running with hyperparameters:  p/volley.json 	***


#### Convert Policy model to Ind in Neat 

In [2]:
import json
import numpy as np
import gym, os
import slimevolleygym
import slimevolleygym.mlp as mlp
from slimevolleygym.mlp import games as games
from slimevolleygym.mlp import Model
from slimevolleygym import multiagent_rollout as rollout


# Settings
random_seed = 612
population_size = 128
total_tournaments = 500000
save_freq = 1000


# Log results
logdir = "ga_rotation_je"
if not os.path.exists(logdir):
  os.makedirs(logdir)

def mutate(param, param_count):
  return param + np.random.normal(size=param_count) * 0.1

curr_policy = Model(games['slimevolleylite'])
prev_policy = Model(games['slimevolleylite'])

/var/folders/nn/nbsb8w3570zfgs23h2s0cdzm0000gn/T/ipykernel_83916/869475039.py:26: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  curr_policy = Model(games['slimevolleylite'])
/var/folders/nn/nbsb8w3570zfgs23h2s0cdzm0000gn/T/ipykernel_83916/869475039.py:27: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prev_policy = Model(games['slimevolleylite'])


Get mutation using Ind class, then convert back to policy model 

In [9]:
# Slime Environment Specific model -> Ind convertion function 
from neat_src import Ind, initIndiv, getNodeOrder, getLayer, LayeredWANN

temp_shapes = [(3, 1), (1, 2)] # debug getlayer functional variable 
node, conn = initIndiv(temp_shapes)
indiv = Ind(conn=conn, node=node)
indiv.express()

connG, nodeG, innov = indiv.mutAddNode(indiv.conn, indiv.node, None, 1, hyp) # add Node mutation 
# connG, innov = indiv.mutAddConn(indiv.conn, indiv.node, None, 1, hyp) # add Connection mutation 

# Convert connG & nodeG into policy model 
# Get shape, weight and bias 

In [10]:
indiv = Ind(conn=connG, node=nodeG)
indiv.express()

True

In [14]:
from neat_src.ind import calculate_layers

dag = {"node": node[0].astype(int).tolist(), "edge": [tuple(c.astype(int)) for c in conn[1:3].T]}
layers = calculate_layers(dag)

print("Node layers:", layers)

Node layers: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 2, 24: 2, 25: 2, 26: 2, 27: 2, 28: 2, 29: 2, 30: 2, 31: 2, 35: 2, 23: 3, 32: 4, 33: 4, 34: 4}
